In [ ]:
import json
import math
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

### Data loading

In [ ]:
dataset_path = "../Dataset/Flow signals/flow_signals_3.json"

In [ ]:
feature_name = "U"
naca_numbers = ['maximum_camber', 'maximum_camber_position', 'maximum_thickness']

dataset = []
with open(dataset_path, 'r') as dataset_file:
  samples = json.load(dataset_file)
  for sample in samples:
    dataset.append({
        "features": sample["features"][feature_name],
        "labels": list(sample["naca_numbers"].values())
    })

### Building the model

In [ ]:
def biuldModel(input_shape):
  # Sequential model - CNN 1D
  model = keras.Sequential([
    keras.layers.InputLayer(input_shape=input_shape),
    keras.layers.Conv1D(filters=256, kernel_size=8, activation=tf.nn.relu),
    keras.layers.AveragePooling1D(pool_size=2),
    keras.layers.Dropout(0.05),
    keras.layers.Conv1D(filters=128, kernel_size=8, activation=tf.nn.relu),
    keras.layers.AveragePooling1D(pool_size=2),
    keras.layers.Dropout(0.05),
    keras.layers.Conv1D(filters=64, kernel_size=8, activation=tf.nn.relu),
    keras.layers.AveragePooling1D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation=tf.nn.relu),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(len(naca_numbers))
  ])

  # Compiling the model
  model.compile(loss='mse', optimizer='adam', metrics=['mae'])
  
  return model

### Distribution statistics

In [ ]:
# Extracting the labels from the dataset
labels = np.array([sample["labels"] for sample in dataset])

# Computing the distribution statistics of the labels
naca_stats = {}
for i in range(len(naca_numbers)):
    max = np.max(labels[:,i])
    min = np.min(labels[:,i])

    naca_stats[naca_numbers[i]] = {
        "min": min,
        "25%": 0.25 * max,
        "50%": 0.5 * max,
        "75%": 0.75 * max,
        "max": max
    }

### Experiments

In [ ]:
epochs = 200 # Number of training epochs
split_percentage = 0.8 # Training and test set split percentage
dataset_size = len(dataset) # Total number of samples available
num_intervals = len(naca_stats[naca_numbers[0]].keys()) - 1 # Number of NACA intervals 
num_experiments = len(naca_numbers) * num_intervals # Total number of experiments
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10) # Early stopping with a patience of 10 epochs

In [ ]:
experiments_results = []

# Iterating over the NACA numbers
for i in range(len(naca_numbers)):

    # Extracting the distibution values of the NACA numbers
    naca_values = [naca_value[1] for naca_value in list(naca_stats[naca_numbers[i]].items())]

    # Iterating over the distribution ranges
    for j in range(len(naca_values) - 1):
        # Extracting the range of values of the NACA numbers for the i-th esperiment
        range_values = (naca_values[j], naca_values[j+1])

        # Training and Test set
        train_dataset = [sample for sample in dataset if sample["labels"][i] < range_values[0] or sample["labels"][i] > range_values[1]]
        test_dataset = [sample for sample in dataset if sample["labels"][i] >= range_values[0] and sample["labels"][i] <= range_values[1]]

        # Extracting the features
        train_features = np.array([sample["features"] for sample in train_dataset])
        test_features = np.array([sample["features"] for sample in test_dataset])

        # Extracting the labels
        train_labels = np.array([sample["labels"] for sample in train_dataset])
        test_labels = np.array([sample["labels"] for sample in test_dataset])

        # Normalizing the data
        mean = train_features.mean(axis=0)
        std = train_features.std(axis=0)

        normalized_train_data = (train_features - mean) / std
        normalized_test_data = (test_features - mean) / std

        # Expanding the dimensions of the training and test features
        normalized_train_features = np.expand_dims(normalized_train_data, axis=2)
        normalized_test_features = np.expand_dims(normalized_test_data, axis=2)

        # Building the model
        input_shape = [np.shape(normalized_train_features)[1], np.shape(normalized_train_features)[2]]
        model = biuldModel(input_shape=input_shape)

        # Training the model using the samples of the i-th experiment
        model.fit(
            normalized_train_features, 
            train_labels,
            epochs=epochs,
            validation_split = 0.2,
            verbose = 0,
            callbacks=[early_stopping]
        )

        # Extracting the values of loss, mean absolute error and mean square error for the i-th experiment
        loss, mae = model.evaluate(normalized_test_features, test_labels, verbose = 0)

        # Print status
        experiment_number = (i + 1) * (j + 1)
        print(f'Experiment {experiment_number}/{num_experiments} | NACA number excluded: {naca_numbers[i]} - {np.around(range_values, 3)} | Training samples: {train_features.shape[0]} | Loss: {round(loss, 6)} | MAE: {round(mae, 6)}')

        # Saving the results of the current experiment into a dictionary
        experiments_results.append({"naca_number": naca_numbers[i], "range_values": range_values, "results": [loss, mae]})

### Results

In [ ]:
# Function to plot results with Bar chart
def plotResults(naca_number, experiments_results):
  fig = plt.figure()
  ax = fig.add_axes([0,0,1,1])

  range_values = [f'{round(result["range_values"][0], 3), round(result["range_values"][1], 3)}' for result in experiments_results]
  data = [experiment_result["results"][1] for experiment_result in experiments_results]

  ax.bar(range_values, data)

  plt.xlabel("Test set range of values of NACA number")
  plt.ylabel("Mean Absolute Error")
  plt.title(naca_number.replace("_", " "))

  plt.show()

In [ ]:
# Plotting the results
for naca_number in naca_numbers:
    result = [experiment_result for experiment_result in experiments_results if experiment_result["naca_number"] == naca_number]
    plotResults(naca_number, result)